In [11]:
from prometheus_pandas import query
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Get current UTC time
current_time = datetime.now(pytz.utc)

# Set end time as current time
end = current_time.strftime('%Y-%m-%dT%H:%M:%SZ')

# Set start time as 5 minutes before the current time
start = (current_time - timedelta(minutes=5)).strftime('%Y-%m-%dT%H:%M:%SZ')

p = query.Prometheus('http://r4-infrastructure-prometheus-server.ricplt.svc.cluster.local:80')

# Loop through each node
for node in range(1, 6):
    node_name = f'node{node}'

    # Query for CPU usage for the specific node
    cpu_query = f'sum by (namespace) (rate(container_cpu_usage_seconds_total{{kubernetes_io_hostname="{node_name}"}}[1m]))'
    df_pod_cpu = p.query_range(cpu_query, start, end, '10s')
    df_pod_cpu['node'] = node_name
    df_pod_cpu.to_csv(f'{start}-out-pod-cpu-{node_name}.csv')



    #Query for Memory usage for the specific node (if needed)
    #Uncomment below lines if you also need memory data per node
    mem_query = f'sum by (namespace) (container_memory_working_set_bytes{{kubernetes_io_hostname="{node_name}"}})'
    df_pod_mem = p.query_range(mem_query, start, end, '10s')
    df_pod_mem.to_csv(f'{start}-out-pod-mem-{node_name}.csv')
